In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from os import listdir
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans, SpectralClustering
import lasio
from sklearn.metrics import mean_absolute_error as mae
import itertools

In [ ]:
las_files = listdir('init/1/las_train2/')

In [ ]:
las = {}
for las_file in las_files[:]:
    las[las_file[:-4]] = (lasio.read('init/1/las_train2/{}'.format(las_file))).df()

In [ ]:
def stats_add(logs, stats):
    new_cols = []
    for log in logs:
        for stat in stats:
            new_cols.append(log + '_' + stat)
    return new_cols

In [ ]:
stats = ['mean', 'median', 'min', 'max']
logs = ['GK', 'BK']#, 'KGL']#'SP']#, 'IK', 'GZ1']
new_cols = stats_add(logs, stats)

In [ ]:
def extract_las_feat(las, logs, stats):
    new_cols = stats_add(logs, stats)
    df_las_feat = pd.DataFrame(np.zeros((len(las), len(new_cols))),
                              index = list(las.keys()),
                              columns = new_cols
                              )
    #anoms = {}
    #conc_df = []
    bad_wells = []
    i=1
    for well in las.keys():
        km_anom = KMeans(n_clusters=2)
    #sc_anom = SpectralClustering(n_clusters=2, n_neighbors=500)
    #log_scaler = MinMaxScaler()

        X_temp = las[well].loc[2500:, logs].dropna()
        if all(X_temp.any()):
        #X = pd.DataFrame(log_scaler.fit_transform(X_temp[X_temp[logs[0]] >0].ewm(span=50).mean()),
        #                 index = X_temp[X_temp[logs[0]] >0].index,
         #                columns = X_temp.columns
         #               )
            X = X_temp[X_temp[logs[0]] >0].ewm(span=50).mean()
        #conc_df.append(X.values)
    
           
            df_anom = pd.DataFrame(km_anom.fit_predict(X),
                                   index = X.index,
                                   columns = ['anom']
                                   )
            X = X.join(df_anom)
            for log in logs:
                if X.loc[(X['anom']==1), 'anom'].count() > X.loc[(X['anom']==0), 'anom'].count():
                    anom_value = 0
                    df_las_feat.loc[well, 'thick'] = X.loc[(X['anom']==0), 'anom'].count()/10
                else:
                    anom_value = 1
                    df_las_feat.loc[well, 'thick'] = X.loc[(X['anom']==1), 'anom'].count()/10
                df_las_feat.loc[well, log+'_mean'] = X.loc[(X['anom']==anom_value) & (X[logs[0]]>0), log].mean()
                df_las_feat.loc[well, log+'_median'] = X.loc[(X['anom']==anom_value) & (X[logs[0]]>0), log].median()
                df_las_feat.loc[well, log+'_min'] = X.loc[(X['anom']==anom_value) & (X[logs[0]]>0), log].min()
                df_las_feat.loc[well, log+'_max'] = X.loc[(X['anom']==anom_value) & (X[logs[0]]>0), log].max()
   
        else:
            bad_wells.append(well)
        i += 1
        print('well {} is done'.format(i))
    return df_las_feat, bad_wells

In [ ]:
df_las_feat, bad_wells = extract_las_feat(las, logs, stats)

In [ ]:
df_las_feat.to_csv('./init/las_feat_train.csv')

In [ ]:
del las

In [ ]:
well = '02dba139f57f4aa50c3fbf56f586551f'
anoms[well][logs+['anom']].plot()
#anoms[well][logs].plot()
#las[well][logs].plot()
ax = plt.gca()
#ax.plot(las[well][logs], c='y')
ax.set_xlim(anoms[well].index.min(), anoms[well].index.max())
#ax.set_ylim(0, 1.2)
ax.legend()

In [ ]:
def extract_las_feat_full(las, logs, stats):
    df_las_feat = pd.DataFrame(np.zeros((len(las), len(new_cols))),
                              index = list(las.keys()),
                              columns = new_cols
                              )
    anoms = {}
    conc_df = []
    bad_wells = []
    i=1
    for well in las.keys():
        km_anom = KMeans(n_clusters=2)
    #sc_anom = SpectralClustering(n_clusters=2, n_neighbors=500)
        log_scaler = MinMaxScaler()

        X_temp = las[well].loc[2000:, logs].dropna()
        if all(X_temp.any()):
        #anoms[well] = pd.DataFrame(log_scaler.fit_transform(X_temp[X_temp[logs[0]] >0].ewm(span=50).mean()),
        #                 index = X_temp[X_temp[logs[0]] >0].index,
         #                columns = X_temp.columns
         #               )
            anoms[well] = X_temp[X_temp[logs[0]] >0].ewm(span=50).mean()
        #conc_df.append(anoms[well].values)
    
           
            df_anom = pd.DataFrame(km_anom.fit_predict(anoms[well]),
                               index = anoms[well].index,
                               columns = ['anom']
                               )
            anoms[well] = anoms[well].join(df_anom)
            for log in logs:
                if anoms[well].loc[(anoms[well]['anom']==1), 'anom'].count() > anoms[well].loc[(anoms[well]['anom']==0), 'anom'].count():
                    anom_value = 0
                    df_las_feat.loc[well, 'thick'] = anoms[well].loc[(anoms[well]['anom']==0), 'anom'].count()/10
                else:
                    anom_value = 1
                    df_las_feat.loc[well, 'thick'] = anoms[well].loc[(anoms[well]['anom']==1), 'anom'].count()/10
                df_las_feat.loc[well, log+'_mean'] = anoms[well].loc[(anoms[well]['anom']==anom_value) & (anoms[well][logs[0]]>0), log].mean()
                df_las_feat.loc[well, log+'_median'] = anoms[well].loc[(anoms[well]['anom']==anom_value) & (anoms[well][logs[0]]>0), log].median()
                df_las_feat.loc[well, log+'_min'] = anoms[well].loc[(anoms[well]['anom']==anom_value) & (anoms[well][logs[0]]>0), log].min()
                df_las_feat.loc[well, log+'_max'] = anoms[well].loc[(anoms[well]['anom']==anom_value) & (anoms[well][logs[0]]>0), log].max()
   
        else:
            bad_wells.append(well)  
        i += 1
        print('well {} is done'.format(i))
    return df_las_feat, bad_wells